In [12]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

#Function to prompt the user to provide the filename.
def get_filename():
    
    while True:
        filename = input('Import a csv or an excel file: ')
        if os.path.exists(filename):
            if filename.split('.')[-1].lower() == 'csv' or filename.split('.')[-1].lower() == 'xlsx':
                return filename
            else:
                print('Invalid file format. Please try again')
        else:
            print(f'Error: There is no file called: {filename}. Please try again.')
    
#Funtion to read the file based on its extension.    
def read_file(filename,file_extension):
    
    if file_extension == 'csv':
        while True:
            delim = input('Please enter the delimiter of the csv file. It must be either ";" or ",": ')
            if delim == ',' or delim == ';':
                return pd.read_csv(filename, delimiter = delim)
            else:
                print('Invalid delimiter. Please try again.')
    else:
        return pd.read_excel(filename)

#Function that fills the missing values with the mean of the variables
def fill_missing_values(df):
    features_df = df.iloc[:, :-1]
    target_df = df.iloc[:, -1]
    features_df = features_df.fillna(features_df.mean())
    df_filled = pd.concat([features_df, target_df], axis=1)
    return df_filled   

#Function that removes the duplicate instances
def remove_duplicates(data):
    df_without_duplicates = data.drop_duplicates()
    return df_without_duplicates

#Function that removes the outliers
def remove_outliers_iqr(df, iqr_factor=1.5):
    conditions = []
    for col in df.columns[:-1]:
        lower_bound = df[col].quantile(5 / 100)
        upper_bound = df[col].quantile(95 / 100)
        condition = (df[col] < lower_bound) | (df[col] > upper_bound)
        conditions.append(condition)
        
    combined_condition = ~pd.concat(conditions, axis=1).any(axis=1)
    return df[combined_condition]

#Function that standardizes the data
def data_standardization(df):
    scaler = StandardScaler()
    for column in df.columns[:-1]:
        df[column] = scaler.fit_transform(df[[column]])[:, 0]
    return df

#Function that balances the data using the Synthetic over sampling technique (SMOTE)
def balance_with_smote(df):
    X = df.drop(df.columns[-1], axis=1)
    y = df[df.columns[-1]]
    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)
    df_balanced = pd.concat([pd.DataFrame(X_resampled), y_resampled], axis=1)
    df_balanced.columns = df.columns
    return df_balanced

#Function that performs filter based feature selection based the f-statistic and chooses the top 10 features
def feature_selection(training_data):
    constant_features = training_data.columns[training_data.nunique() == 1]
    training_data.drop(constant_features, axis=1, inplace=True)
    X = training_data.drop(training_data.columns[-1], axis=1)
    y = training_data[training_data.columns[-1]]
    f_values, p_values = f_classif(X, y)
    features_importance = pd.Series(f_values, index=X.columns)
    features_importance_sorted = features_importance.sort_values(ascending=False)
    best_features = features_importance_sorted.index[:10]
    return training_data[list(best_features) + [training_data.columns[-1]]]

#Function to split the data
def data_split(data_frame):
    X = data_frame.iloc[:, :-1]
    y = data_frame.iloc[:, -1]
    unique_classes = y.unique()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    training_data = pd.concat([X_train, y_train], axis=1)
    testing_data = pd.concat([X_test, y_test], axis=1)
    training_data = data_preprocess_for_the_supervised_algorithms(training_data, 1)
    return training_data, testing_data

#Function for the preprocess of the data
def data_preprocess_for_the_supervised_algorithms(data, flag):
    if flag == 0:
        data = fill_missing_values(data)
        data = remove_duplicates(data)
        data = remove_outliers_iqr(data)
        data = data_standardization(data)
        data = feature_selection(data)
    if flag == 1:
        data = balance_with_smote(data)
    return data
    
#Function that implements the Logistic regression classification algorithm
def logistic_regression(training_data, testing_data, random_state=None):
    while True:
        try:
            test_size = input('Specify the test size (0.0,0.5] (We recommend to not use a test size greater than 0.2): ')
            test_size = float(test_size)
            if test_size > 0.0 and test_size <= 0.5:
                break
            else:
                print('Invalid test size.')
                continue
        except:
            print('Invalid input. Please try again')
    X_train, y_train = training_data.iloc[:, :-1], training_data.iloc[:, -1]
    X_test, y_test = testing_data.iloc[:, :-1], testing_data.iloc[:, -1]
    unique_classes = y_train.unique()
    X_train = X_train.values
    X_test = X_test.values
    
    model = LogisticRegression(random_state=random_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = round((accuracy_score(y_test, y_pred)*100),2)
    precision = round((precision_score(y_test, y_pred, labels=unique_classes, average='micro')*100),2)
    recall = round((recall_score(y_test, y_pred, labels=unique_classes, average='micro')*100),2)
    cm = confusion_matrix(y_test, y_pred, labels=unique_classes)
    f1 = round((f1_score(y_test, y_pred, average='micro')*100),3)
    
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:\n", f1)
    print("Confusion Matrix:\n", cm)
    
filename = get_filename()
file_extension = filename.split('.')[-1].lower()
data_frame = read_file(filename,file_extension)
data_frame = data_preprocess_for_the_supervised_algorithms(data_frame, 0)
training_data, testing_data = data_split(data_frame)
logistic_regression(training_data, testing_data)

Import a csv or an excel file:  4.csv
Please enter the delimiter of the csv file. It must be either ";" or ",":  ,
Specify the test size (0.0,0.5] (We recommend to not use a test size greater than 0.2):  0.2


0.2
Accuracy: 86.0
Precision: 86.0
Recall: 86.0
F1-Score:
 86.0
Confusion Matrix:
 [[51 10]
 [11 78]]
